In [9]:
%load_ext autoreload
%autoreload 2

In [13]:
# Always reload changed modules
%load_ext autoreload
%autoreload 2

import importlib, inspect
import src.data.loader as loader
importlib.reload(loader)          # force reload once

print("Using loader from:", loader.__file__)
print(inspect.getsource(loader.load_raw))  # sanity: shows the exact code the notebook is using

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using loader from: /Users/saab/Documents/DSBA/Interviews/Alki/alki-forecast/src/data/loader.py
def load_raw(path: str) -> pd.DataFrame:
    """
    Load raw data from a CSV file. Ensure required columns are present
    and parse dates.

    Args:
        path (str): Path to the CSV file.
    
    Returns:
        pd.DataFrame: Cleaned DataFrame with correct types.

    """
    # Read; keep_default_na=True makes 'nan','NaN','N/A' => NaN
    df = pd.read_csv(path, sep=";", dtype={"CUSTOMER": "string"})

    # Schema check
    missing = [c for c in REQUIRED_COLUMNS if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    # Parse types
    df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce", dayfirst=True)
    df["QUANTITY"] = pd.to_numeric(df["QUANTITY"], errors="coerce")

    # 1) drop fully empty rows (often trailing garbage)
    df = df.dropna

In [15]:
from src.data.loader import load_raw, reindex_daily
from src.data.validation import validate_raw, print_validation_report

In [16]:
df = load_raw("data/raw/train set.csv")       # strict load
rep, ok = validate_raw(df)                # build report
print_validation_report(rep, ok)          # pretty print

# (optional) daily index for EDA-only visuals
df_daily = reindex_daily(df)

      CUSTOMER  rows      start        end  duplicates_same_date  nan_qty  negative_qty  nonfinite_qty  noninteger_qty  missing_days  max_gap_days
       ARGALYS   830 2019-08-01 2023-06-02                     0        0             0              0               0           572            22
LES MIRACULEUX   950 2019-05-10 2023-06-02                     0        0             0              0               0           535           208
  MINCI DELICE  1531 2017-09-30 2023-06-02                     0        0             0              0               0           541             4
    NUTRAVANCE  2014 2015-08-26 2023-06-02                     0        0             0              0               0           824            13
        GLOBAL  5325 2015-08-26 2023-06-02                     0        0             0              0               0          2472           208

Validation: PASS


In [17]:
df_daily.head()

,DATE,CUSTOMER,QUANTITY
0,2019-08-01,ARGALYS,6.0
1,2019-08-02,ARGALYS,NaN
2,2019-08-03,ARGALYS,NaN
3,2019-08-04,ARGALYS,NaN
4,2019-08-05,ARGALYS,NaN
